In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
import glob
import os

# Load Dataset

In [2]:
trainDataDir = "dataset/two-sigma-connect-rental-listing-inquiries/trainCleanedOutlier.json.zip"

trainData = pd.read_json(trainDataDir, convert_dates=['created'])
trainData = trainData.reset_index(drop=True)
trainData

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,2.0,3,b66d3340ba269e6f184bfad550a6f05f,2016-06-24 07:55:11,Welcome to your new home! This extravagantly o...,president street,"[Dining Room, Laundry in Building, High Speed ...",low,40.6678,7211226,-73.9398,929baf387cf71d98482a9743ab8a0890,[https://photos.renthop.com/2/7211226_7c7b8bc2...,3050,1475 president street
1,2.0,3,b66d3340ba269e6f184bfad550a6f05f,2016-04-29 03:29:11,"<![CDATA[3 bedrooms, 3100, Bedford Stuyvesant/...",president street,[],low,40.6678,6941286,-73.9398,a12c733c6cba53a6d2fe8a1fdfd5a674,[https://photos.renthop.com/2/6941286_252212fd...,3100,1475 president street
2,1.0,1,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-17 01:22:28,"This newly renovated, queen sized 1 Bed / 1 Ba...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7174518,-73.9810,befc71000fbd97ffe17b1ef8566b0590,[https://photos.renthop.com/2/7174518_ac2cde58...,2850,504 east 12th street
3,1.0,3,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-08 01:12:34,"Prime East Village 3 bedroom, 1 bath apartment...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7122279,-73.9810,e7f859b81a2a75ac93042a721d70a0aa,[https://photos.renthop.com/2/7122279_fea77391...,4200,504 east 12th street
4,1.0,1,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-11 01:27:45,Welcome to your next new home.This 1BR/1BA uni...,east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7138528,-73.9810,89179891e92a87e72ac3132fc5a977e8,[https://photos.renthop.com/2/7138528_940f2e70...,2850,504 east 12th street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43523,1.0,3,92bbbf38baadfde0576fc496bd41749c,2016-04-05 03:58:33,There is 700 square feet of recently renovated...,west 171 street,"[Elevator, Dishwasher, Hardwood Floors]",low,40.8433,6824800,-73.9396,a61e21da3ba18c7a3d54cfdcc247e1f8,[https://photos.renthop.com/2/6824800_0682be16...,2800,620 west 171 street
43524,1.0,2,5565db9b7cba3603834c4aa6f2950960,2016-04-02 02:25:31,"2 bedroom apartment with updated kitchen, rece...",broadway,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",medium,40.8198,6813268,-73.9578,8f90e5e10e8a2d7cf997f016d89230eb,[https://photos.renthop.com/2/6813268_1e6fcc32...,2395,3333 broadway
43525,1.0,1,67997a128056ee1ed7d046bbb856e3c7,2016-04-26 05:42:03,No Brokers Fee * Never Lived 1 Bedroom 1 Bathr...,210 brighton 15th street,"[Dining Room, Elevator, Pre-War, Laundry in Bu...",medium,40.5765,6927093,-73.9554,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/6927093_93a52104...,1850,210 brighton 15th street
43526,1.0,2,3c0574a740154806c18bdf1fddd3d966,2016-04-19 02:47:33,Wonderful Bright Chelsea 2 Bedroom apartment o...,west 21st street,"[Pre-War, Laundry in Unit, Dishwasher, No Fee,...",medium,40.7448,6892816,-74.0017,c3cd45f4381ac371507090e9ffabea80,[https://photos.renthop.com/2/6892816_1a8d087a...,4195,350 west 21st street


# Image Data

In [3]:
imageDir = "dataset/two-sigma-connect-rental-listing-inquiries/images_sample"

img_data = pd.DataFrame(columns = ['listing_id', 'grayscale_hist', 'avg_pixel_values'])

for file in glob.glob(imageDir + '/*/*.jpg'):
    img = Image.open(file, 'r')
    img_grayscale = img.convert('L')
    img_resize = img.resize((64, 64), Image.ANTIALIAS)
    
    file_name = os.path.basename(file)
    listing_id = file_name.split('_')[0]
    
    grayscale_hist = img_grayscale.histogram()
    pixel_values = list(img_resize.getdata())
    
    R_pixel_values = list(img_resize.getdata(band=0))
    G_pixel_values = list(img_resize.getdata(band=1))
    B_pixel_values = list(img_resize.getdata(band=2))
    
    avg_pixel_values = (np.array(R_pixel_values) + np.array(G_pixel_values) + np.array(B_pixel_values)) / 3
    
    img_data = img_data.append({'listing_id': listing_id, 
                                'grayscale_hist': grayscale_hist,
                                'avg_pixel_values': avg_pixel_values
                               }, ignore_index=True)
    
    
img_data['listing_id'] = pd.to_numeric(img_data['listing_id'])
img_data

,listing_id,grayscale_hist,avg_pixel_values
0,6811957,"[0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 3, 4, ...","[203.0, 201.0, 201.0, 200.0, 199.0, 198.0, 196..."
1,6811957,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 6, 5, 19, 53, 49, ...","[193.33333333333334, 195.33333333333334, 198.3..."
2,6811957,"[1, 0, 1, 2, 17, 7, 13, 14, 14, 28, 28, 25, 36...","[188.0, 189.0, 189.0, 189.66666666666666, 190...."
3,6811957,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[173.0, 171.66666666666666, 170.66666666666666..."
4,6811957,"[0, 0, 0, 0, 0, 0, 0, 4, 4, 6, 3, 4, 3, 9, 11,...","[216.0, 214.0, 214.0, 213.0, 212.0, 212.0, 211..."
...,...,...,...
409,6812263,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[159.66666666666666, 161.33333333333334, 159.3..."
410,6812264,"[9, 35, 107, 169, 243, 410, 492, 629, 715, 781...","[170.33333333333334, 177.33333333333334, 181.0..."
411,6812264,"[9, 10, 15, 12, 11, 16, 13, 10, 10, 8, 7, 3, 9...","[183.66666666666666, 183.66666666666666, 182.6..."
412,6812266,"[0, 0, 0, 0, 0, 5, 25, 39, 72, 208, 442, 813, ...","[104.0, 105.0, 105.33333333333333, 106.6666666..."


In [4]:
trainData = trainData.merge(img_data, on='listing_id', how='left')
# also include number of photos
trainData["num_photos"] = trainData["photos"].apply((lambda x: 0 if not x else len(x)))
trainData

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address,grayscale_hist,avg_pixel_values,num_photos
0,2.0,3,b66d3340ba269e6f184bfad550a6f05f,2016-06-24 07:55:11,Welcome to your new home! This extravagantly o...,president street,"[Dining Room, Laundry in Building, High Speed ...",low,40.6678,7211226,-73.9398,929baf387cf71d98482a9743ab8a0890,[https://photos.renthop.com/2/7211226_7c7b8bc2...,3050,1475 president street,NaN,NaN,8
1,2.0,3,b66d3340ba269e6f184bfad550a6f05f,2016-04-29 03:29:11,"<![CDATA[3 bedrooms, 3100, Bedford Stuyvesant/...",president street,[],low,40.6678,6941286,-73.9398,a12c733c6cba53a6d2fe8a1fdfd5a674,[https://photos.renthop.com/2/6941286_252212fd...,3100,1475 president street,NaN,NaN,10
2,1.0,1,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-17 01:22:28,"This newly renovated, queen sized 1 Bed / 1 Ba...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7174518,-73.9810,befc71000fbd97ffe17b1ef8566b0590,[https://photos.renthop.com/2/7174518_ac2cde58...,2850,504 east 12th street,NaN,NaN,3
3,1.0,3,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-08 01:12:34,"Prime East Village 3 bedroom, 1 bath apartment...",east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7122279,-73.9810,e7f859b81a2a75ac93042a721d70a0aa,[https://photos.renthop.com/2/7122279_fea77391...,4200,504 east 12th street,NaN,NaN,6
4,1.0,1,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-11 01:27:45,Welcome to your next new home.This 1BR/1BA uni...,east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7138528,-73.9810,89179891e92a87e72ac3132fc5a977e8,[https://photos.renthop.com/2/7138528_940f2e70...,2850,504 east 12th street,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43637,1.0,3,92bbbf38baadfde0576fc496bd41749c,2016-04-05 03:58:33,There is 700 square feet of recently renovated...,west 171 street,"[Elevator, Dishwasher, Hardwood Floors]",low,40.8433,6824800,-73.9396,a61e21da3ba18c7a3d54cfdcc247e1f8,[https://photos.renthop.com/2/6824800_0682be16...,2800,620 west 171 street,NaN,NaN,5
43638,1.0,2,5565db9b7cba3603834c4aa6f2950960,2016-04-02 02:25:31,"2 bedroom apartment with updated kitchen, rece...",broadway,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",medium,40.8198,6813268,-73.9578,8f90e5e10e8a2d7cf997f016d89230eb,[https://photos.renthop.com/2/6813268_1e6fcc32...,2395,3333 broadway,NaN,NaN,5
43639,1.0,1,67997a128056ee1ed7d046bbb856e3c7,2016-04-26 05:42:03,No Brokers Fee * Never Lived 1 Bedroom 1 Bathr...,210 brighton 15th street,"[Dining Room, Elevator, Pre-War, Laundry in Bu...",medium,40.5765,6927093,-73.9554,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/6927093_93a52104...,1850,210 brighton 15th street,NaN,NaN,3
43640,1.0,2,3c0574a740154806c18bdf1fddd3d966,2016-04-19 02:47:33,Wonderful Bright Chelsea 2 Bedroom apartment o...,west 21st street,"[Pre-War, Laundry in Unit, Dishwasher, No Fee,...",medium,40.7448,6892816,-74.0017,c3cd45f4381ac371507090e9ffabea80,[https://photos.renthop.com/2/6892816_1a8d087a...,4195,350 west 21st street,NaN,NaN,5


In [5]:
trainData[trainData['grayscale_hist'].notna()]

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address,grayscale_hist,avg_pixel_values,num_photos
114,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-02 01:13:38,**NO FEE**MIDTOWN EAST**D/W**Elevator** Doorma...,east 49th street,"[Doorman, Fitness Center, No Fee, Dogs Allowed...",low,40.7539,6812054,-73.9677,38e613cd90ba43943211be10168ee175,[https://photos.renthop.com/2/6812054_bc8af2f4...,3725,333 east 49th street,"[78, 141, 146, 123, 189, 359, 554, 776, 795, 8...","[36.666666666666664, 38.666666666666664, 41.33...",3
115,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-02 01:13:38,**NO FEE**MIDTOWN EAST**D/W**Elevator** Doorma...,east 49th street,"[Doorman, Fitness Center, No Fee, Dogs Allowed...",low,40.7539,6812054,-73.9677,38e613cd90ba43943211be10168ee175,[https://photos.renthop.com/2/6812054_bc8af2f4...,3725,333 east 49th street,"[0, 0, 0, 0, 1, 5, 8, 26, 39, 42, 29, 33, 40, ...","[73.33333333333333, 63.666666666666664, 78.666...",3
116,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-02 01:13:38,**NO FEE**MIDTOWN EAST**D/W**Elevator** Doorma...,east 49th street,"[Doorman, Fitness Center, No Fee, Dogs Allowed...",low,40.7539,6812054,-73.9677,38e613cd90ba43943211be10168ee175,[https://photos.renthop.com/2/6812054_bc8af2f4...,3725,333 east 49th street,"[0, 0, 0, 0, 0, 0, 2, 5, 30, 48, 32, 34, 70, 8...","[22.0, 21.0, 21.0, 22.0, 23.333333333333332, 2...",3
1208,1.0,1,f51e3f6b4886bb775ab9a6a0f34ff0ef,2016-04-02 01:21:29,Beautiful one bedroom apartment like no other!...,east 69th street,"[Doorman, Dogs Allowed, Cats Allowed]",low,40.7675,6812258,-73.9607,5511bb6c502eaf88b51bac57f7ad0be7,[https://photos.renthop.com/2/6812258_a093d1c2...,3807,219 east 69th street,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 1, 6, ...","[73.0, 73.0, 74.0, 71.0, 70.66666666666667, 74...",4
1209,1.0,1,f51e3f6b4886bb775ab9a6a0f34ff0ef,2016-04-02 01:21:29,Beautiful one bedroom apartment like no other!...,east 69th street,"[Doorman, Dogs Allowed, Cats Allowed]",low,40.7675,6812258,-73.9607,5511bb6c502eaf88b51bac57f7ad0be7,[https://photos.renthop.com/2/6812258_a093d1c2...,3807,219 east 69th street,"[0, 0, 10, 61, 120, 228, 260, 236, 259, 301, 3...","[202.66666666666666, 203.66666666666666, 204.3...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41648,1.0,1,bc4e62116277654d4df66ab77a1152f8,2016-04-01 23:26:07,Reduced Fee!! Priced To Rent!\rLarge Newly Upd...,west end avenue,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",medium,40.7939,6811973,-73.9738,262471a6a678adb458e879b092b23b4b,[https://photos.renthop.com/2/6811973_c87c8f6d...,2745,700 west end avenue,"[0, 0, 19, 191, 511, 498, 291, 211, 185, 235, ...","[117.0, 119.33333333333333, 124.0, 125.3333333...",5
41649,1.0,1,bc4e62116277654d4df66ab77a1152f8,2016-04-01 23:26:07,Reduced Fee!! Priced To Rent!\rLarge Newly Upd...,west end avenue,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",medium,40.7939,6811973,-73.9738,262471a6a678adb458e879b092b23b4b,[https://photos.renthop.com/2/6811973_c87c8f6d...,2745,700 west end avenue,"[0, 0, 0, 6, 7, 31, 39, 47, 48, 33, 26, 38, 27...","[141.33333333333334, 152.33333333333334, 137.6...",5
41650,1.0,1,bc4e62116277654d4df66ab77a1152f8,2016-04-01 23:26:07,Reduced Fee!! Priced To Rent!\rLarge Newly Upd...,west end avenue,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",medium,40.7939,6811973,-73.9738,262471a6a678adb458e879b092b23b4b,[https://photos.renthop.com/2/6811973_c87c8f6d...,2745,700 west end avenue,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 14, 7, 13, 1...","[132.66666666666666, 140.0, 140.66666666666666...",5
41805,1.0,0,104bfeddd65a0890b071c3a09cf81704,2016-04-01 22:56:00,"This Enormous Studio Features: Harwood Floors,...",east 54th street,"[Cats Allowed, Dogs Allowed, No Fee, Laundry I...",medium,40.7576,6811965,-73.9677,3b630ec9cb6eee53b92cfac7f42e3bf4,[https://photos.renthop.com/2/6811965_b8f942e6...,2000,230 east 54th stree

# Text Data

### Feature Extraction

In [6]:
trainData['desc_word_count'] = trainData['description'].apply(lambda x: len(str(x).split(" ")))
trainData['desc_char_count'] = trainData['description'].str.len()

trainData[['desc_word_count', 'desc_char_count']]

,desc_word_count,desc_char_count
0,183,1276
1,44,372
2,32,224
3,62,390
4,134,756
...,...,...
43637,131,852
43638,70,552
43639,81,550
43640,118,845


In [7]:
def avg_word(sentence):
    words = sentence.split()
    if (len(words) == 0):
        return 0
    
    return (sum(len(word) for word in words)/len(words))

trainData['desc_avg_word_length'] = trainData['description'].apply(lambda x: avg_word(x))
trainData[['desc_avg_word_length']]

,desc_avg_word_length
0,6.044199
1,7.477273
2,6.225806
3,5.393443
4,4.684211
...,...
43637,5.596899
43638,6.971014
43639,5.949367
43640,6.385965


In [8]:
def count_non_alpha_num(str):
    return sum(1 - (s.isalpha() or s == " ") for s in str)

trainData['desc_special_char_count'] = trainData['description'].apply(lambda x: count_non_alpha_num(x))
trainData[['description', 'desc_special_char_count']]

,description,desc_special_char_count
0,Welcome to your new home! This extravagantly o...,66
1,"<![CDATA[3 bedrooms, 3100, Bedford Stuyvesant/...",86
2,"This newly renovated, queen sized 1 Bed / 1 Ba...",30
3,"Prime East Village 3 bedroom, 1 bath apartment...",15
4,Welcome to your next new home.This 1BR/1BA uni...,22
...,...,...
43637,There is 700 square feet of recently renovated...,46
43638,"2 bedroom apartment with updated kitchen, rece...",64
43639,No Brokers Fee * Never Lived 1 Bedroom 1 Bathr...,74
43640,Wonderful Bright Chelsea 2 Bedroom apartment o...,67


In [9]:
trainData['desc_num_count'] = trainData['description'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
trainData['desc_upper_count'] = trainData['description'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
trainData[['description', 'desc_num_count', 'desc_upper_count']]

,description,desc_num_count,desc_upper_count
0,Welcome to your new home! This extravagantly o...,3,2
1,"<![CDATA[3 bedrooms, 3100, Bedford Stuyvesant/...",1,3
2,"This newly renovated, queen sized 1 Bed / 1 Ba...",2,0
3,"Prime East Village 3 bedroom, 1 bath apartment...",2,1
4,Welcome to your next new home.This 1BR/1BA uni...,2,2
...,...,...,...
43637,There is 700 square feet of recently renovated...,1,0
43638,"2 bedroom apartment with updated kitchen, rece...",1,0
43639,No Brokers Fee * Never Lived 1 Bedroom 1 Bathr...,2,3
43640,Wonderful Bright Chelsea 2 Bedroom apartment o...,1,0


### Data Preprocessing

#### Convert to lower case and remove punctuation

In [10]:
trainData['description'] = trainData['description'].apply(lambda x: " ".join(x.lower() for x in x.split()))
trainData['description'] = trainData['description'].str.replace('[^\w\s]','')

trainData[['description']]

,description
0,welcome to your new home this extravagantly ou...
1,cdata3 bedrooms 3100 bedford stuyvesantbushwic...
2,this newly renovated queen sized 1 bed 1 bath...
3,prime east village 3 bedroom 1 bath apartment ...
4,welcome to your next new homethis 1br1ba unit ...
...,...
43637,there is 700 square feet of recently renovated...
43638,2 bedroom apartment with updated kitchen recen...
43639,no brokers fee never lived 1 bedroom 1 bathro...
43640,wonderful bright chelsea 2 bedroom apartment o...


#### Remove common and rare words

In [11]:
word_counts = pd.Series(' '.join(trainData['description']).split()).value_counts()

freq = word_counts[:10]
rare = word_counts[-10:]
freq_list = list(freq.index)
rare_list = list(rare.index)

trainData['description'] = trainData['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_list and x not in rare_list))

print(freq_list)
print(rare_list)
trainData[['description']]

['and', 'br', 'the', 'a', 'to', 'in', 'with', 'of', 'is', 'this']
['179659', 'designsone', 'avenuebrbravailable', 'metropolitanlorimer', 'sportconveniently', 'deliveriessoak', 'marbkle', '23bdefcapathfor', '89thstreet', 'studiorenovatedexposed']


,description
0,welcome your new home extravagantly outfitted ...
1,cdata3 bedrooms 3100 bedford stuyvesantbushwic...
2,newly renovated queen sized 1 bed 1 bath unit ...
3,prime east village 3 bedroom 1 bath apartment ...
4,welcome your next new homethis 1br1ba unit loc...
...,...
43637,there 700 square feet recently renovated space...
43638,2 bedroom apartment updated kitchen recently i...
43639,no brokers fee never lived 1 bedroom 1 bathroo...
43640,wonderful bright chelsea 2 bedroom apartment o...


#### Remove stop words

In [12]:
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
display(stopwords.words('english'))

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [13]:
stop_list = stopwords.words('english')
trainData['description'] = trainData['description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_list))

trainData[['description']]

,description
0,welcome new home extravagantly outfitted 3 bed...
1,cdata3 bedrooms 3100 bedford stuyvesantbushwic...
2,newly renovated queen sized 1 bed 1 bath unit ...
3,prime east village 3 bedroom 1 bath apartment ...
4,welcome next new homethis 1br1ba unit located ...
...,...
43637,700 square feet recently renovated space beaut...
43638,2 bedroom apartment updated kitchen recently i...
43639,brokers fee never lived 1 bedroom 1 bathroom p...
43640,wonderful bright chelsea 2 bedroom apartment q...


#### Count number of features for each listing

In [14]:
trainData['num_features'] = trainData['features'].apply(lambda x: len(x))

#### Extract year, month, date to their individual columns

In [15]:
trainData['year'] = trainData['created'].dt.year
trainData['month'] = trainData['created'].dt.month
trainData['day'] = trainData['created'].dt.day

In [16]:
trainData

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,...,desc_word_count,desc_char_count,desc_avg_word_length,desc_special_char_count,desc_num_count,desc_upper_count,num_features,year,month,day
0,2.0,3,b66d3340ba269e6f184bfad550a6f05f,2016-06-24 07:55:11,welcome new home extravagantly outfitted 3 bed...,president street,"[Dining Room, Laundry in Building, High Speed ...",low,40.6678,7211226,...,183,1276,6.044199,66,3,2,8,2016,6,24
1,2.0,3,b66d3340ba269e6f184bfad550a6f05f,2016-04-29 03:29:11,cdata3 bedrooms 3100 bedford stuyvesantbushwic...,president street,[],low,40.6678,6941286,...,44,372,7.477273,86,1,3,0,2016,4,29
2,1.0,1,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-17 01:22:28,newly renovated queen sized 1 bed 1 bath unit ...,east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7174518,...,32,224,6.225806,30,2,0,3,2016,6,17
3,1.0,3,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-08 01:12:34,prime east village 3 bedroom 1 bath apartment ...,east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7122279,...,62,390,5.393443,15,2,1,3,2016,6,8
4,1.0,1,7e9fd0dea8ad2c8ac0a1022a1f0d997b,2016-06-11 01:27:45,welcome next new homethis 1br1ba unit located ...,east 12th street,"[Pre-War, Dogs Allowed, Cats Allowed]",low,40.7287,7138528,...,134,756,4.684211,22,2,2,3,2016,6,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43637,1.0,3,92bbbf38baadfde0576fc496bd41749c,2016-04-05 03:58:33,700 square feet recently renovated space beaut...,west 171 street,"[Elevator, Dishwasher, Hardwood Floors]",low,40.8433,6824800,...,131,852,5.596899,46,1,0,3,2016,4,5
43638,1.0,2,5565db9b7cba3603834c4aa6f2950960,2016-04-02 02:25:31,2 bedroom apartment updated kitchen recently i...,broadway,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",medium,40.8198,6813268,...,70,552,6.971014,64,1,0,7,2016,4,2
43639,1.0,1,67997a128056ee1ed7d046bbb856e3c7,2016-04-26 05:42:03,brokers fee never lived 1 bedroom 1 bathroom p...,210 brighton 15th street,"[Dining Room, Elevator, Pre-War, Laundry in Bu...",medium,40.5765,6927093,...,81,550,5.949367,74,2,3,11,2016,4,26
43640,1.0,2,3c0574a740154806c18bdf1fddd3d966,2016-04-19 02:47:33,wonderful bright chelsea 2 bedroom apartment q...,west 21st street,"[Pre-War, Laundry in Unit, Dishwasher, No Fee,...",medium,40.7448,6892816,...,118,845,6.385965,67,1,0,5,2016,4,19


In [17]:
trainDataTextExtract = "dataset/two-sigma-connect-rental-listing-inquiries/trainTextExtract.json.zip"
trainData.to_json(trainDataTextExtract)